In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\anush\anaconda3\Lib\runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\anush\anaconda3\Lib\runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "C:\Users\anush\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\anush\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\anush\anaconda3\Lib\site-pac

AttributeError: _ARRAY_API not found

In [3]:
data = [
    [1,5,7],
    [1,5,8],
    [1,6,8]
]

columns = ['A','B','C']

df_data = pd.DataFrame(data, columns=columns)
print(df_data)

   A  B  C
0  1  5  7
1  1  5  8
2  1  6  8


In [4]:
from src.database.db_connection import *

In [5]:
engine = create_engine_for_db()

In [6]:
df_data.to_sql('test_prototype_1',engine,schema='prototype1',index=False,if_exists='replace')

3

In [7]:
fd_constraints = [(['A'],['B','C'])]

# Step 1 - simplify FDs to have one attribute at RHS

In [8]:
from src.utilities.step1_simplify_fds import *

In [9]:
fd_constraints = simplify_FDs(fd_constraints)

In [10]:
fd_constraints

[(['A'], ['B']), (['A'], ['C'])]

# Step 2: clusters + generating possible repairs per cluster

In [11]:
from src.utilities.step2_clustering_and_generating_repairs import *

### Step 2_0: Preprocess data by adding a new column with unique identifier

In [12]:
df_data = preprocess_data(df_data)

In [13]:
df_data

,A,B,C,uuid
0,1,5,7,1
1,1,5,8,2
2,1,6,8,3


### Step 2_1: Take user input for:
### 1) Delete costs
### 2) Update costs
### 3) Epsilon value: small constant to avoid division by zero
### 4) Score Name: UNIFORM or COST_BASED calculations for probability values 
### 5) Constraint hardness: An Array to define the hardness of each constraint. Each element of the array will have value between [0,1.0]

In [14]:
delete_cost = 2
update_cost = 1
eps=1e-9
prob_type = 'UNIFORM'
constraint_hardness = [1.0,1.0]

### Step 2_2: Compute actions per cluster for each FD

In [15]:
from src.utilities.step2_clustering_and_generating_repairs import *

In [16]:
actions,cluster_ids,rv_probs = compute_actions(df_data, fd_constraints, constraint_hardness,delete_cost, update_cost,eps,prob_type)

In [17]:
rv_probs

{'fd1B1=1': 0.08333333333333333,
 'fd1B1=2': 0.08333333333333333,
 'fd1B1=3': 0.08333333333333333,
 'fd1B1=4': 0.08333333333333333,
 'fd1B1=5': 0.08333333333333333,
 'fd1B1=6': 0.08333333333333333,
 'fd1B1=7': 0.08333333333333333,
 'fd1B1=8': 0.08333333333333333,
 'fd1B1=9': 0.08333333333333333,
 'fd1B1=10': 0.08333333333333333,
 'fd1B1=11': 0.08333333333333333,
 'fd1B1=12': 0.08333333333333333,
 'fd2C1=1': 0.08333333333333333,
 'fd2C1=2': 0.08333333333333333,
 'fd2C1=3': 0.08333333333333333,
 'fd2C1=4': 0.08333333333333333,
 'fd2C1=5': 0.08333333333333333,
 'fd2C1=6': 0.08333333333333333,
 'fd2C1=7': 0.08333333333333333,
 'fd2C1=8': 0.08333333333333333,
 'fd2C1=9': 0.08333333333333333,
 'fd2C1=10': 0.08333333333333333,
 'fd2C1=11': 0.08333333333333333,
 'fd2C1=12': 0.08333333333333333}

### Step 2_3 Combine repair actions per cluster

### Step 2_4 Pick Minimum cost repair actions out of all per cluster

In [18]:
all_actions = min_cost_actions(cluster_ids,actions,delete_cost,update_cost)

### Step 2_5 Apply repair actions to the orignal data

In [19]:
repaired_versions =  apply_combined_actions(df_data, all_actions, uuid_col="uuid")

In [20]:
repaired_versions = pd.concat(repaired_versions, ignore_index=True)

In [21]:
repaired_versions = repaired_versions.drop(columns=["uuid"])

In [22]:
rv_probabilities = filter_and_condition_min_cost_rvs(rv_probs, repaired_versions)

# Step 3: Merge same rows by combining sentences

In [23]:
repaired_versions

,A,B,C,_sentences
0,1,5,7,fd1B1=4 & fd2C1=4
1,1,5,7,fd1B1=4 & fd2C1=4
2,1,5,7,fd1B1=4 & fd2C1=4
3,1,5,8,fd1B1=4 & fd2C1=8
4,1,5,8,fd1B1=4 & fd2C1=8
5,1,5,8,fd1B1=4 & fd2C1=8
6,1,5,8,fd1B1=4 & fd2C1=12
7,1,5,8,fd1B1=4 & fd2C1=12
8,1,5,8,fd1B1=4 & fd2C1=12
9,1,5,7,fd1B1=8 & fd2C1=4


In [24]:
from src.utilities.step4_group_similar_sentences import *

In [25]:
repaired_df = group_similar_rows_together(repaired_versions)

In [26]:
repaired_df

,A,B,C,_sentences
0,1,5,7,(fd1B1=4 & fd2C1=4) | (fd1B1=8 & fd2C1=4)
1,1,5,8,(fd1B1=4 & fd2C1=8) | (fd1B1=4 & fd2C1=12) | (...
2,1,6,8,(fd1B1=12 & fd2C1=8) | (fd1B1=12 & fd2C1=12)


In [27]:
rv_probabilities

{'fd1B1=4': 0.3333333333333333,
 'fd1B1=8': 0.3333333333333333,
 'fd1B1=12': 0.3333333333333333,
 'fd2C1=4': 0.3333333333333333,
 'fd2C1=8': 0.3333333333333333,
 'fd2C1=12': 0.3333333333333333}

# Step 5: Insert in Dubio

In [28]:
drop_stmt = f"DROP table prototype1.test_prototype_1_prob CASCADE;"
with engine.begin() as connection:
    connection.execute(text(drop_stmt))

In [29]:
repaired_df.to_sql('test_prototype_1_prob',engine,schema='prototype1',index=False,if_exists='replace')

3

In [30]:
update_stmt_dic = f"UPDATE prototype1._dict set  dict = dictionary('') where name='prototype_dict';"
with engine.begin() as connection:
    connection.execute(text(update_stmt_dic))

In [31]:
entries = [f"{k}:{v}" for k, v in rv_probabilities.items()]
joined = ';'.join(entries)

update_stmt = f"UPDATE prototype1._dict\nSET dict=add(dict, '{joined}')\nWHERE name='prototype_dict';"

print(update_stmt)

UPDATE prototype1._dict
SET dict=add(dict, 'fd1B1=4:0.3333333333333333;fd1B1=8:0.3333333333333333;fd1B1=12:0.3333333333333333;fd2C1=4:0.3333333333333333;fd2C1=8:0.3333333333333333;fd2C1=12:0.3333333333333333')
WHERE name='prototype_dict';


In [32]:
with engine.begin() as connection:
    connection.execute(text(update_stmt))